In [7]:
import ee
import os
import pandas

# Initialize Earth Engine
ee.Initialize()

In [8]:
# Read in relevant data and print properties of a feature collection
modis_evi = ee.ImageCollection('MODIS/006/MOD13Q1').select(['EVI']) 
states = ee.FeatureCollection('ft:181cSJpsQXc9Gvh1w8Qm8FVtuawyHh_LlhfQwf0N0') 
western_states = states.filter(ee.Filter.eq('REGION10', 4)).filter(ee.Filter.inList('NAME10', ['Alaska', 'Hawaii']).Not()) 
ecoregions = ee.FeatureCollection('ft:1Ec8IWsP8asxN-ywSqgXWMuBaxI6pPaeh6hC64lA').filterBounds(western_states) 

print(ecoregions.aggregate_array('ECO_NAME').getInfo())

[u'Northern mixed grasslands', u'South Central Rockies forests', u'South Central Rockies forests', u'Montana Valley and Foothill grasslands', u'Northern short grasslands', u'North Central Rockies forests', u'Montana Valley and Foothill grasslands', u'South Central Rockies forests', u'Montana Valley and Foothill grasslands', u'South Central Rockies forests', u'Wyoming Basin shrub steppe', u'Snake-Columbia shrub steppe', u'Great Basin shrub steppe', u'South Central Rockies forests', u'South Central Rockies forests', u'South Central Rockies forests', u'Palouse grasslands', u'Okanagan dry forests', u'British Columbia mainland coastal forests', u'Cascade Mountains leeward forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'Puget lowland forests', u'

In [9]:
# Define function to convert EE feature collection to Pandas dataframe
def fc2df(fc):
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        attr = f['properties']
        dictarr.append(attr)  
    df = pandas.DataFrame(dictarr)
    return df 

july_evi_mean = modis_evi.filterDate('2012-07-01','2012-08-01').mean()
ecos_july_evi = july_evi_mean.reduceRegions(ecoregions,reducer=ee.Reducer.mean(),scale=250).select(['ECO_NAME','mean'])
ecos_july_evi_df = fc2df(ecos_july_evi)

print(ecos_july_evi_df)



                                      ECO_NAME         mean
0                    Northern mixed grasslands  4500.794453
1                South Central Rockies forests  2853.477095
2                South Central Rockies forests  2610.081111
3       Montana Valley and Foothill grasslands  1945.670554
4                    Northern short grasslands  2456.547943
5                North Central Rockies forests  3388.873326
6       Montana Valley and Foothill grasslands  3068.395972
7                South Central Rockies forests  2702.460070
8       Montana Valley and Foothill grasslands  1996.414112
9                South Central Rockies forests  3157.991563
10                  Wyoming Basin shrub steppe  1393.951387
11                 Snake-Columbia shrub steppe  1618.749349
12                    Great Basin shrub steppe  1305.214594
13               South Central Rockies forests  2597.873164
14               South Central Rockies forests  1882.146666
15               South Central Rockies f

In [11]:
directory = '/content/output'
if not os.path.exists(directory):
    os.makedirs(directory)

# Write the results to a CSV
ecos_july_evi_df.to_csv(os.path.join(directory, 'ecos_july_evi_df.csv'))